In [1]:
from pyxnat import Interface
import xnat
import os, glob
import pydicom
import tempfile
from tqdm import tqdm
import shutil
from zipfile import ZipFile, ZIP_DEFLATED
import pandas as pd

In [ ]:
# #The URL of your XNAT
# xnat_host = os.environ['XNAT_HOST']

# #A temporary token which can be used as a username
# user = os.environ['XNAT_USER']

# #A temporary token which can be used as a password
# pwd = os.environ['XNAT_PASS']

In [ ]:
# session = xnat.connect(xnat_host, user=user, password=pwd)

In [2]:
session = xnat.connect('https://xnat.crmbm.univ-amu.fr', user='hdary')

In [3]:
project = "VIDA_BRAIN_EPINOV"
path_data_cemerem = '/DATA_CEMEREM/data/protocols/vida/brain/epinov/'

In [5]:
project = "TERRA_BRAIN_EPINOV"
path_data_cemerem = '/DATA_CEMEREM/data/protocols/terra/brain/epinov/'

In [6]:
my_project = session.projects[project]

Check what is on XNAT and data_cemerem

In [7]:
in_data_cemerem = glob.glob(path_data_cemerem + '*')
in_data_cemerem = [os.path.basename(x) for x in in_data_cemerem]
# in_data_cemerem = [x for x in in_data_cemerem if os.path.isdir(x)]
# in_data_cemerem = [glob.glob(x + '/*/*/*.dcm')[0] for x in in_data_cemerem]



In [ ]:
in_data_cemerem_corrected = [i.replace('_', '-').lower() for i in in_data_cemerem]

In [ ]:
in_data_cemerem_corrected = [i.replace('_', '-').lower() for i in in_data_cemerem]
dico_cemerem = {}
for i, j in enumerate(in_data_cemerem_corrected):
    dico_cemerem[j] = in_data_cemerem[i]

In [ ]:
in_xnat = my_project.subjects.values()
in_xnat = [x.label for x in in_xnat]

In [10]:
in_xnat_corrected = [i.replace('_', '-').lower() for i in in_xnat]
dico_xnat = {}
for i, j in enumerate(in_xnat_corrected):
    dico_xnat[j] = in_xnat[i]

In [11]:
subjects_list = list(set(in_xnat_corrected + in_data_cemerem_corrected))

In [13]:
list_data_cemerem = {}
list_in_xnat = {}
list_to_add_to_xnat = {}

for s in subjects_list:
    if s in in_data_cemerem_corrected:
        list_data_cemerem[s] = True
    else:
        in_data_cemerem_corrected[s] = False
    if s in in_xnat_corrected:
        list_in_xnat[s] = True
    else:
        list_in_xnat[s] = False
    if s in in_xnat_corrected and s in in_data_cemerem_corrected:
        list_to_add_to_xnat[s] = False
    elif s not in in_xnat_corrected and s in in_data_cemerem_corrected:
        list_to_add_to_xnat[s] = True
    elif s in in_xnat_corrected and s not in in_data_cemerem_corrected:
        list_to_add_to_xnat[s] = 'ERROR'


In [14]:
data = []
for subject in subjects_list:
    data.append({'subject': subject, 'in XNAT': list_in_xnat[subject], 'in DATA_CEMEREM': list_data_cemerem[subject], 'To add to XNAT': list_to_add_to_xnat[subject]})

df = pd.DataFrame(data)

In [15]:
df

,subject,in XNAT,in DATA_CEMEREM,To add to XNAT
0,01-51-l-l,True,True,False
1,01-82-b-a,True,True,False
2,01-79-g-s,True,True,False
3,01-80-v-t,True,True,False
4,01-59-p-v,True,True,False
5,01-55-l-m,True,True,False
6,01-63-b-k,True,True,False
7,01-49-rd,True,True,False
8,01-54-d-r,True,True,False
9,01-47-q-m,True,True,False


In [16]:
filtered_df = df[df['To add to XNAT'] == True]

In [17]:
filtered_df

,subject,in XNAT,in DATA_CEMEREM,To add to XNAT


In [18]:
list(filtered_df['subject'])

[]

fezfzef

check tag dicom for naming **session** : (0020,0052)

In [19]:
def get_session(data):
    session = data[0x020,0x052].value
    return session

check tag dicom Patient Name for naming **subject** : (0010,0010)

In [20]:
def get_subject_name(data):
    subject = data[0x010,0x010].value
    return subject

temporary zip dicom folder

In [23]:
def zip_data(folder):
    try:
        dossier_temporaire = tempfile.mkdtemp()
        fichier_zip_temporaire = os.path.join(dossier_temporaire, 'archive.zip')

        with ZipFile(fichier_zip_temporaire, 'w', ZIP_DEFLATED) as zip_file:
            for dirpath, dirs, files in os.walk(folder):
                for f in files:
                    zip_file.write(os.path.join(dirpath, f),
                                   os.path.relpath(os.path.join(dirpath, f),
                                                   os.path.dirname(folder)))
                    
        # shutil.make_archive(fichier_zip_temporaire, 'zip', folder)
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")
    
    return fichier_zip_temporaire


In [ ]:
def map_demographics(my_project, subject, experiemnt):

    s = my_project.subjects[subject]
    print(s.id, s.label)
    e = s.experiments[experiemnt]
    
    scan = e.scans[0]
    file_obj = scan.resources['DICOM'].files[0]

    with file_obj.open() as fin:
        data = pydicom.dcmread(fin, stop_before_pixels=True)
        patient_birth_date = data[0x010,0x030].value
        sex = data[0x010,0x040].value
        age = data[0x010,0x1010].value
        size = data[0x010,0x1020].value
        weight = data[0x010,0x1030].value

        try:
            s.demographics.age = int(age.replace('Y',''))
        except ValueError as e:
            print('Error setting age:', e)
            
        try:
            s.demographics.yob = patient_birth_date
        except ValueError as e:
            print('Error setting year of birth:', e)

        try:
            s.demographics.gender = sex.replace('M','male').replace('F','female')
        except ValueError as e:
            print('Error setting gender:', e)

Upload demographics values

Create subject and session in XNAT

In [25]:
for su in tqdm(list(filtered_df['subject'])):
    print('subject', su)

    print('getting dicom tag info')
    path = glob.glob(path_data_cemerem + '/' + dico_cemerem[su] + '/*/*')[0]
    path = glob.glob(path + '/*.dcm')[0]
    data = pydicom.dcmread(path, stop_before_pixels=True)
    session_dicom = str(get_session(data).split('176082.2.')[-1][:12])
    subject_dicom = str(get_subject_name(data))
    print('session_dicom', session_dicom)
    print('subject_dicom', subject_dicom)

    print('creating subject and session')
    # if you want to create subject and session
    subject_xnat = session.classes.SubjectData(parent=my_project, label=subject_dicom)
    # session_xnat = session.classes.MrSessionData(parent=subject_xnat, label=session_dicom)

    print('zipping data'')
    zipped_data = zip_data(path_data_cemerem + '/' + dico_cemerem[su])

    print('uploading data')
    prearchive_session = session.services.import_(zipped_data, project=project, 
                                                  subject=subject_dicom, experiment=session_dicom,
                                                  destination='/prearchive')
    
    # experiment = prearchive_session.archive(subject=subject_dicom, experiment=session_dicom)
    print('uploading demographics')
    map_demographics(my_project, subject_dicom, session_dicom)

    print('deleting zipped data')
    shutil.rmtree(os.path.dirname(zipped_data))


202009181323
01_33_D-S


for a specific subject 

In [ ]:
path1 = '/DATA_CEMEREM/data/protocols/vida/brain/epinov/01-33-d-s/20200918/00_0001_aahead-scout'
path = glob.glob(path1 + '/*.dcm')[0]
data = pydicom.dcmread(path, stop_before_pixels=True)
session_dicom = str(get_session(data).split('176082.2.')[-1][:12])
subject_dicom = str(get_subject_name(data))
print(session_dicom)
print(subject_dicom)

# if you want to create subject and session
subject_xnat = session.classes.SubjectData(parent=my_project, label=subject_dicom)
session_xnat = session.classes.MrSessionData(parent=subject_xnat, label=session_dicom)
zipped_data = zip_data(path1)

prearchive_session = session.services.import_(zipped_data, project=project, 
                                                subject=subject_dicom, experiment=session_dicom,
                                                destination='/prearchive')

# experiment = prearchive_session.archive(subject=subject_dicom, experiment=session_dicom)
shutil.rmtree(os.path.dirname(zipped_data))

In [27]:
session.prearchive.sessions()

[]